In [4]:
import tensorflow as tf
from tensorflow.keras.callbacks import ModelCheckpoint
import numpy as np
from sklearn.utils import shuffle

import sys 
sys.path.append('../src/utils/')
sys.path.append('../src/models/')
sys.path.append('../src/visualization/')

from particles import Photon
from center_finder import CenterFinder

%load_ext autoreload
%autoreload 2

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


# Load and prepare data for training from one- and two-particle samples

In [5]:
photon_1 = Photon(n_pcl=1)
photon_2 = Photon(n_pcl=2)

photons = [photon_1, photon_2]

In [126]:
# define the desired model of the seed finder for data pre-processing
seed_paths = {"model_path": "../notebooks/models/seed_finder/run_3/", 
              "weight_path": "../notebooks/models/seed_finder/run_3/weights.27-0.0105.hdf5"}

In [127]:
X, y = photon_2.data_for_center_finder("debug", 0.3, **seed_paths)

100%|██████████| 1000/1000 [00:00<00:00, 3228.85it/s]


1040/1040 [==============================] - 3s 3ms/step


100%|██████████| 950/950 [00:00<00:00, 40245.53it/s]


In [144]:
# combine elements for 1 and 2 photon samples into one dataset
Xtrain, ytrain = ([], [], []), {'center':[], 'energy': [], 'seed': []} 
Xvalid, yvalid = ([], [], []), {'center':[], 'energy': [], 'seed': []} 

for photon in photons:
    Xi, yi = photon.data_for_center_finder("debug", 0.3, **seed_paths)
    [Xtrain[i].extend(Xi[i]) for i in range(3)]
    {ytrain[i].extend(yi[i]) for i in ['center', 'energy', 'seed']}

    Xi, yi = photon.data_for_center_finder("debug", 0.3, **seed_paths)
    [Xvalid[i].extend(Xi[i]) for i in range(3)]
    yvalid['center'].extend(yi['center']), yvalid['energy'].extend(yi['energy']), yvalid['seed'].extend(yi['seed'])

Xtrain = [np.array(Xtrain[i]) for i in range(3)]
ytrain = {i: np.array(ytrain[i]) for i in ['center', 'energy', 'seed']}

Xvalid = [np.array(Xvalid[i]) for i in range(3)]

100%|██████████| 1000/1000 [00:00<00:00, 5077.41it/s]


1094/1094 [==============================] - 3s 3ms/step


100%|██████████| 1000/1000 [00:00<00:00, 4878.22it/s]


1094/1094 [==============================] - 3s 3ms/step


100%|██████████| 1000/1000 [00:00<00:00, 3693.80it/s]


1040/1040 [==============================] - 3s 3ms/step


100%|██████████| 1000/1000 [00:00<00:00, 3839.80it/s]


1040/1040 [==============================] - 3s 3ms/step


100%|██████████| 950/950 [00:00<00:00, 40189.51it/s]


In [146]:
ytrain['center'].shape, Xtrain[0].shape

((1950, 4, 2), (1950, 7, 7, 4))